In [14]:
import numpy as np
import os
import pandas as pd
from numpy.ma.extras import average
"""_vehicle_output 파일에서 데이터 가공 (차종별 평균, 가중 평균속도)"""
folder_path = r"C:\VISSIM_Workspace\network_new\test"
mer_name = "network01_mer"

file_path = os.path.join(folder_path, mer_name, "vehicle_output")
vehicle_files = sorted([f for f in os.listdir(file_path) if f.endswith(".xlsx")], key=lambda x: int(x.split("_")[0]) if x.split("_")[0].isdigit() else 0)
print("vehicle_files : ", vehicle_files)
for n in range(len(vehicle_files)):
    df = pd.read_excel(os.path.join(file_path, vehicle_files[n]))
    df["NewMeasurem"] = df["Measurem."] % 1000

    vehicle_type_list = [100, 300, 630, 640, 650]
    results = []
    for i in range(len(vehicle_type_list)):
        filtered_df = df[df["Vehicle type"] == vehicle_type_list[i]]
        # 피벗 테이블 생성
        old_df = filtered_df.pivot_table(
            index="time",
            columns=["NewMeasurem", "Measurem."],
            values="v[km/h]",
            aggfunc="mean"
        )
        # 결과를 저장할 딕셔너리
        combined_cols = {}
        print(old_df)


        # level=0("NewMeasurem") 별로 그룹핑 후 2개 열의 데이터를 이어 붙인 후 새로운 컬럼 생성
        for measurem_group, cols in old_df.groupby(axis=1, level=0):
            if len(cols.columns) == 2:
                col1, col2 = cols.columns
                new_col_name = f'{col1[1]}'

                # 1번째 열과 2번째 열을 하나로 합친다.
                combined_series = pd.concat([cols[col1], cols[col2]], ignore_index=True)
                combined_cols[new_col_name] = combined_series
        combined_df = pd.DataFrame(combined_cols)


        # 평균속도
        avg_speed = combined_df.mean(axis=0)
        # 표준편차
        std = combined_df.std(ddof=0)
        print(std)
        # 변동계수
        vrt_num = std / avg_speed
        # 가중치
        add_per =  1/vrt_num

        for j in range(len(avg_speed)):
            results.append({
                "Vehicle type": vehicle_type_list[i],
                "NewMeasurem": int(list(avg_speed.index)[j]) % 1000,
                "AverageSpeed": avg_speed.iloc[j],
                "Weight (가중치)": add_per.iloc[j]
            })
    weights_df = pd.DataFrame(results)
    #print(weights_df[(weights_df["Vehicle type"] == 630) & (weights_df["NewMeasurem"] == 2)])
    ####### 가중 평균속도 계산 #######

    # 평균속도 * 가중치
    weights_df["Weighted Value"] = weights_df["AverageSpeed"] * weights_df["Weight (가중치)"]

    # NewMeasurem별로 그룹화 및 합 계산
    sum_weighted = weights_df.groupby("NewMeasurem")["Weighted Value"].sum()
    sum_weights = weights_df.groupby("NewMeasurem")["Weight (가중치)"].sum()

    # 가중 평균 속도 계산
    final_df = (sum_weighted / sum_weights).reset_index(name="Weighted Avg Speed")
    #print(final_df)
    final_df = final_df.set_index("NewMeasurem").T
    final_df.index = ["가중평균속도"]  # 이게 첫 번째 행(인덱스)로 들어감


    # 차종별 평균속도 계산 (NewMeasurem 기준)
    vehicle_avg = weights_df.groupby(["NewMeasurem", "Vehicle type"])["AverageSpeed"].mean().unstack()
    vehicle_avg = vehicle_avg.reindex(columns=vehicle_type_list)  # 차종 순서 고정
    vehicle_avg = vehicle_avg
    vehicle_avg.index.name = "NewMeasurem"
    vehicle_avg.columns = [f"차종{col}" for col in vehicle_avg.columns]

    # 가중평균속도와 결합
    final_df = pd.concat([vehicle_avg.T, final_df])
    excel_dir = os.path.join(folder_path, mer_name, "weights_output")
    """
    if not os.path.exists(excel_dir):
        os.makedirs(excel_dir)
    last_file = xlsx_files = sorted([f for f in os.listdir(excel_dir) if f.endswith(".xlsx")], key=lambda x: int(x.split("_")[0]) if x.split("_")[0].isdigit() else 0)

    if not xlsx_files: # 폴더에 파일이 존재하지 않을 때의 시작 번호
        last_file = 1
    else:
        last_file = int(xlsx_files[-1].split("_")[0]) + 1  # 가장 마지막 파일 번호 + 1
    excel_path = os.path.join(excel_dir, f"{last_file}_weights_output.xlsx")
    final_df.to_excel(excel_path, index=True)
    print(f"생성파일: {excel_path}")
    """

vehicle_files :  ['1_vehicle_output.xlsx']
NewMeasurem     1               2               3               4            \
Measurem.     10001   20001   10002   20002   10003   20003   10004   20004   
time                                                                          
1860         106.47  107.07  106.10  106.68  105.95  105.35  106.09  106.07   
1920         107.18  106.93  107.30  106.95  107.39  107.45  106.74  107.26   
1980         106.50  106.37  106.13  106.26  105.02  106.08  105.17  106.16   
2040         106.49  105.12  106.24  104.98  106.74  104.96  106.90  104.86   
2100         108.07  105.67  107.49  106.05  106.80  106.15  106.44  106.13   
2160         104.95  104.47  105.68  104.39  105.81  104.29  106.05  103.98   
2220         105.80  105.74  105.67  105.12  106.13  105.40  105.46  105.74   
2280         106.23  105.91  105.93  106.37  106.80  105.62  106.99  105.30   
2340         106.88  105.33  106.63  105.56  107.02  105.11  106.84  105.24   
2400     